In [ ]:
import numpy as np

# %matplotlib widget

# Needs this line or else pandas plots do not show in jupyterlab
%matplotlib inline

from numpy import squeeze

from scipy.integrate import solve_ivp, cumtrapz
from scipy.linalg import null_space, expm, inv, norm, eig
from numpy import trace, eye, newaxis, sum, pi, unwrap, array, zeros, argmax, hstack, asarray, tile, cross, concatenate

from scipy.spatial.transform import Rotation
from scipy.signal import correlate

from matplotlib.pyplot import *
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection, Line3DCollection
from matplotlib.animation import FuncAnimation
import matplotlib.patches as mpatches

import inspect

e1 = array([[1,0,0]]).T
e2 = array([[0,1,0]]).T
e3 = array([[0,0,1]]).T

In [ ]:
def check_equality(arg1,arg2,tol=1e-6):
    max_error = np.amax(abs(arg1-arg2))
    if(max_error > tol):
        caller = inspect.getframeinfo(inspect.stack()[1][0])
        print(f"High state error: tol < {max_error:.2E} \t Line {caller.lineno:3}: {caller.code_context}")
    return

In [ ]:
start_time = 0
last_time = 100

In [ ]:
# start_time = 3.9
# last_time = 15

In [ ]:
# # POINT_MASS = True
# X = np.load('/home/dcabecinhas/20191011_170237_ros_load_controller.log.npy')
# last_time = 30

# Last run
list_logs = !ls ~/.ros/numpy_logs/*.npz
list_logs[-1]

# # Last run
# list_logs = !ls ~/.ros/numpy_logs/*20191026_115007*.npz
# list_logs[-1]

log = np.load(list_logs[-1],allow_pickle=True)

In [ ]:
state_vector = log['state_vector']
sim_gains = log['sim_gains']
sim_parameters = log['sim_parameters']

t = state_vector[-1]

start_idx = round(start_time * 50)
final_idx = (t == 0).argmax() - 2

last_t = (t >= min(last_time,t[final_idx])).argmax()

t = state_vector[-1,start_idx:last_t]
state = state_vector[0:-1,start_idx:last_t]

print("Last t =", state_vector[-1,last_t])

In [ ]:
print(list_logs[-1])
print("delta t (avg) = ", np.mean(np.diff(t)))
print("delta t (med) = ", np.median(np.diff(t)))

In [ ]:
# Restore gains and simulation parameters

(Kp, Kv, X_pv,
L_q, L_oq, kq, koq,
L_qR, L_oqR, kqR, kqo,
L_a, kr, ko) = sim_gains

(POINT_MASS,NO_U_PERP_CROSS_TERMS,TLD_DERIVATIVES_ZERO,
 USE_OPTIMIZATION,REMOVE_INSTABILITY_TERMS,REAL_QUADROTOR,_) = sim_parameters

In [ ]:
(Kp.ravel(), Kv.ravel(), X_pv.ravel(),
L_q, L_oq, kq, koq,
L_qR, L_oqR, kqR, kqo,
L_a, kr, ko)

In [ ]:
# Add src/ directory to path so we can import the same .py files as the real code
import sys
sys.path.append("../src")

if(POINT_MASS):
    from point_mass_4_vehicles import skew, mt, unpack_solution, process_state, n_cables, n_dims, mass_quad, g, state_sizes, rho, lq, mQ, mL, unskew,  length, width, height, compute_TL, pr
    I = np.eye(3)
    Md = np.zeros(3).reshape([-1,3,1])
    M = Md
else:
    from rigid_body_4_vehicles import skew, mt, unpack_solution, process_state, n_cables, n_dims, mass_quad, g, state_sizes, rho, lq, mQ, mL, unskew, I, length, width, height, compute_vector_TL_restrictions_body, compute_vector_TL_restrictions_inertial, pr, Rr

In [ ]:
(p,v,R,o,delta_TLd,q,oq,qR,qo,V) = unpack_solution(state.T)

dy, state = process_state(t,state.T)

In [ ]:
check_equality(p,state.p)
check_equality(v,state.v)
if(not POINT_MASS):
    check_equality(R,state.R)
    check_equality(o,state.o)
check_equality(delta_TLd,state.delta_TLd)
check_equality(q,state.q)
check_equality(oq,state.oq)
check_equality(qR,state.qR)
check_equality(oq,state.oq)

In [ ]:
# Fills current namespace with variables from 'state' namespace
for key,val in state.__dict__.items(): 
    exec(key + '=val')

qp = p + R@rho - q*lq
qv = v + R @ skew(o) @ rho - dq*lq

In [ ]:
(np.trace(np.eye(3) - qR[0], axis1=-1, axis2=-2)).max()

In [ ]:
# pd = np.array([0,0,-0.5]).reshape(-1,3,1)

# pd = np.stack(last_t*[pd]).reshape(-1,1,3,1)
# dpd = 0*pd 
# d2pd = 0*pd

# Rd = np.stack(last_t*[np.identity(3)]).reshape(-1,1,3,3)
# od = np.stack(last_t*[np.zeros(3)]).reshape(-1,1,3,1)
# taud = np.stack(last_t*[np.zeros(3)]).reshape(-1,1,3,1)

In [ ]:
# e3 = np.array([0,0,1]).reshape(-1,1,3,1)

In [ ]:
if(POINT_MASS):
    Rd = np.eye(3)
    od = np.zeros(3).reshape([3,1])
    taud = np.zeros(3).reshape([3,1])

In [ ]:
# TODO: vectorize

# From: https://www.learnopencv.com/rotation-matrix-to-euler-angles/
# Calculates Rotation Matrix given euler angles.
# theta :: [roll pitch yaw]
def eulerAnglesToRotationMatrix(theta):

    R_x = np.array([[1,          0,                 0                ],
                    [0,         np.cos(theta[0]),  -np.sin(theta[0]) ],
                    [0,         np.sin(theta[0]),   np.cos(theta[0]) ]])

    R_y = np.array([[ np.cos(theta[1]),   0,      np.sin(theta[1])  ],
                    [ 0,                  1,      0                 ],
                    [-np.sin(theta[1]),   0,      np.cos(theta[1])  ]])

    R_z = np.array([[np.cos(theta[2]),   -np.sin(theta[2]),    0],
                    [np.sin(theta[2]),    np.cos(theta[2]),    0],
                    [0,                   0,                   1]])

    R = R_z @ R_y @ R_x

    return R

# Checks if a matrix is a valid rotation matrix.
def isRotationMatrix(R) :
#     return True
#     return amax(abs((np.eye(3) - mt(R)@R))) < 1e-4
    return amax(abs((np.eye(3) - mt(R)@R))) < 1e-6

# Calculates rotation matrix to euler angles
# The result is the same as MATLAB except the order
# of the euler angles ( x and z are swapped ).
def rotationMatrixToEulerAngles(R) :

#     assert(isRotationMatrix(R))

    sy = np.sqrt(R[...,0,0] * R[...,0,0] +  R[...,1,0] * R[...,1,0])

    singular = sy < 1e-6

    x = array(np.arctan2(R[...,2,1] , R[...,2,2]))
    y = array(np.arctan2(-R[...,2,0], sy))
    z = array(np.arctan2(R[...,1,0], R[...,0,0]))
    
    
    x[singular] = np.arctan2(-R[singular,1,2], R[singular,1,1])
    y[singular] = np.arctan2(-R[singular,2,0], sy[singular])
    z[singular] = 0

    return np.array([x, y, z])


In [ ]:
# pd = pr(t)
# dpd = dpr(t)
# d2pd = d2pr(t)
# d3pd = d3pr(t)

pe = p-pd
ve = v-dpd
ae = dv-d2pd

if(not POINT_MASS):
    Re = mt(R) @ Rd
    e_R = 1/2 * unskew(mt(Re) - Re)
    e_o = (o - od)

    Re = mt(R) @ Rd
    tau = do
    dtau = d2o
    dod = taud
    e_R = 1/2 * unskew(mt(Re) - Re)
    e_o = (o - od)
    dRe = - skew(o - Re @ od) @ Re
    de_R = 1/2 * unskew( mt(Re) @ skew(o - Re @ od) - mt(skew(o - Re @ od)) @ Re )
    d2e_R = 1/2 * unskew( 
        mt(dRe) @ skew(o - Re @ od) + mt(Re) @ skew(do - dRe @ od - Re @ dod)  
         - mt(skew(do - dRe @ od - Re @ dod)) @ Re - mt(skew(o - Re @ od)) @ dRe 
    )
    de_o = tau - taud
    d2e_o = dtau - dtaud

## Plots

In [ ]:
def y_lim_equalize(axes):

    y_min = 0
    y_max = 0
    for ax in axes:
        y_min = min(y_min, ax.get_ylim()[0])
        y_max = max(y_max, ax.get_ylim()[1])

    for ax in axes:
        ax.set_ylim(y_min,y_max)

In [ ]:
fig, axes = subplots(2, 2, figsize=(10, 8), sharex=True)

all_axes = [ax.grid() for sublist in axes for ax in sublist]

all_axes = [ax for sublist in axes for ax in sublist]

fig.suptitle('V', fontsize=16)

# plot(
#     t, squeeze(1/2 * mt(pe) @ Kp @ pe ),
#     t, squeeze(+ 1/2 * mt(ve) @ ve ),
#     t, squeeze(+ mt(pe) @ X_pv @ ve),
#     t, squeeze(+ L_a * 1/2 * kr * trace(eye(3) - mt(R) @ Rd, axis1=-1, axis2=-2)[...,newaxis,newaxis] ),
#     t, squeeze(+ L_a * 1/2 * mt(e_o) @ e_o ),
#     t, squeeze(+ L_q * sum(1 - mt(q) @ qd, axis=-3, keepdims=True)),
#     t, squeeze(+ L_oq * 1/2 * sum(mt(zoq) @ zoq, axis=-3, keepdims=True))
#     )
# grid()
# legend([
#     'p',
#     'v',
#     'x',
#     'r',
#     'o',
#     'q',
#     'oq'
# ])

if(POINT_MASS):
    all_axes[0].plot(t, squeeze(
    1/2 * mt(pe) @ Kp @ pe 
    + 1/2 * mt(ve) @ ve 
    + mt(pe) @ X_pv @ ve
    + L_q * sum(1 - mt(q) @ qd, axis=-3, keepdims=True)
    + L_oq * 1/2 * sum(mt(zoq) @ zoq, axis=-3, keepdims=True)
))
else:
    all_axes[0].plot(t, squeeze(
    1/2 * mt(pe) @ Kp @ pe 
    + 1/2 * mt(ve) @ ve 
    + mt(pe) @ X_pv @ ve
    + L_a * 1/2 * kr * trace(eye(3) - mt(R) @ Rd, axis1=-1, axis2=-2)[...,newaxis,newaxis] 
    + L_a * 1/2 * mt(e_o) @ e_o 
    + L_q * sum(1 - mt(q) @ qd, axis=-3, keepdims=True)
    + L_oq * 1/2 * sum(mt(zoq) @ zoq, axis=-3, keepdims=True)
))
        
all_axes[0].set_title('V')
all_axes[0].legend([
    'V'])
    
all_axes[1].plot(t, squeeze(1/2 * mt(pe) @ Kp @ pe ),
            t, squeeze(+ 1/2 * mt(ve) @ ve ),
            t, squeeze(+ mt(pe) @ X_pv @ ve))
all_axes[1].set_title('p vs v')
all_axes[1].legend([
    'p',
    'v',
    'x'])

if(not POINT_MASS):
    all_axes[2].plot(t, squeeze(+ L_a * 1/2 * kr * trace(eye(3) - mt(R) @ Rd, axis1=-1, axis2=-2)[...,newaxis,newaxis] ),
                t, squeeze(+ L_a * 1/2 * mt(e_o) @ e_o ))
    all_axes[2].set_title('R vs o')
    all_axes[2].legend([
        'R',
        'o'])

all_axes[3].plot(t, squeeze(+ L_q * sum(1 - mt(q) @ qd, axis=-3, keepdims=True)),
            t, squeeze(+ L_oq * 1/2 * sum(mt(zoq) @ zoq, axis=-3, keepdims=True)))
all_axes[3].set_title('q vs oq')
all_axes[3].legend([
    'q',
    'oq'])

y_lim_equalize(all_axes[1:])

In [ ]:
# dV_ = (
#     - mt(pe) @ Kp @ X_pv @ (pe) 
#     - mt(pe) @ Kv @ X_pv @ (ve)
#     - mt(ve) @ (Kv - X_pv) @ ve 
#     - L_a * ko * mt( e_o ) @ ( e_o )
#     - kq * sum( mt(qd) @ skew(q) @ mt(skew(q)) @ qd , axis=-3, keepdims=True)
#     - koq * sum( mt( zoq ) @ ( zoq ), axis=-3, keepdims=True)
# )

fig, axes = subplots(2, 2, figsize=(10, 8), sharex=True)

all_axes = [ax.grid() for sublist in axes for ax in sublist]

all_axes = [ax for sublist in axes for ax in sublist]

fig.suptitle('dV', fontsize=16)

if(POINT_MASS):
    all_axes[0].plot(t, squeeze(
        - mt(pe) @ Kp @ X_pv @ (pe) 
        - mt(pe) @ Kv @ X_pv @ (ve)
        - mt(ve) @ (Kv - X_pv) @ ve 
        - kq * sum( mt(qd) @ skew(q) @ mt(skew(q)) @ qd , axis=-3, keepdims=True)
        - koq * sum( mt( zoq ) @ ( zoq ), axis=-3, keepdims=True)
    ))
    all_axes[0].set_title('dV')
    all_axes[0].legend([
        'V'])
else:
    all_axes[0].plot(t, squeeze(
        - mt(pe) @ Kp @ X_pv @ (pe) 
        - mt(pe) @ Kv @ X_pv @ (ve)
        - mt(ve) @ (Kv - X_pv) @ ve 
        - L_a * ko * mt( e_o ) @ ( e_o )
        - kq * sum( mt(qd) @ skew(q) @ mt(skew(q)) @ qd , axis=-3, keepdims=True)
        - koq * sum( mt( zoq ) @ ( zoq ), axis=-3, keepdims=True)
    ))
    all_axes[0].set_title('dV')
    all_axes[0].legend([
        'V'])
    
all_axes[1].plot(t, squeeze(- mt(pe) @ X_pv @ Kp @ pe))
all_axes[1].plot(t, squeeze(- mt(ve) @ (Kv - X_pv) @ ve))
all_axes[1].plot(t, squeeze(- mt(pe) @ X_pv @ Kv @ ve))
all_axes[1].set_title('p vs v')
all_axes[1].legend([
    'p',
    'v',
    'x'])

if(not POINT_MASS):
    all_axes[2].plot(t, squeeze( - L_a * ko * mt( e_o ) @ ( e_o ) ))
    all_axes[2].set_title('R vs o')
    all_axes[2].legend([
        'R',
        'o'])

all_axes[3].plot(t, squeeze(- kq * sum( mt(qd) @ skew(q) @ mt(skew(q)) @ qd , axis=-3, keepdims=True)),
            t, squeeze(- koq * sum(mt( zoq ) @ ( zoq ), axis=-3, keepdims=True)))
all_axes[3].set_title('q vs oq')
all_axes[3].legend([
    'q',
    'oq'])

y_lim_equalize(all_axes[1:])

In [ ]:
fig, axes = subplots(7, 4, figsize=(16, 12), sharex=True)

all_axes = [ax.grid() for sublist in axes for ax in sublist]

axes[0,0].plot(t,squeeze(p),
        t,squeeze(pd))
axes[0,0].set_title('p vs pd')

axes[0,1].plot(t,squeeze(v),
        t,squeeze(dpd))
axes[0,1].set_title('v vs dpd')

if(not POINT_MASS):
    axes[0,2].plot(t,squeeze(R.reshape(-1,9)),
            t,squeeze(Rd.reshape(-1,9)))
    axes[0,2].set_title('R vs Rd')

    axes[0,3].plot(t,squeeze(o),
            t,squeeze(od))
    axes[0,3].set_title('o vs od')

###################################

axes[1,0].plot(t, squeeze(Fd[:,:,0:2,:]))
axes[1,0].set_title('Fd')

if(not POINT_MASS):
    axes[1,1].plot(t, squeeze(Md))
    axes[1,1].set_title('Md')

axes[1,2].plot(t, squeeze(TLd.reshape(-1,12)))
axes[1,2].set_title('TLd')

axes[1,3].plot(t, squeeze(mud.reshape(-1,12)))
axes[1,3].set_title('mud')

# ###################################

axes[2,0].plot(t, squeeze(F[:,:,0:2,:]))
axes[2,0].set_title('F')

if(not POINT_MASS):
    axes[2,1].plot(t, squeeze(M))
    axes[2,1].set_title('M')

# axes[2,2].plot(t, squeeze(TL))
# axes[2,2].set_title('TL')

axes[2,3].plot(t, squeeze(mu.reshape(-1,12)))
axes[2,3].set_title('mu')

# ###################################

axes[3,0].plot(t,squeeze(180/np.pi * np.arccos(mt(q) @ qd)))
axes[3,0].set_title('q error (deg)')

axes[3,1].plot(t,squeeze(norm(oq-oqd,axis=-2)))
axes[3,1].set_title('oq error')

axes[3,2].plot(t,squeeze(q.reshape(-1,12)),t,squeeze(qd.reshape(-1,12)))
axes[3,2].set_title('q vs qd')

axes[3,3].plot(t,squeeze(oq.reshape(-1,12)), t, squeeze(oqd.reshape(-1,12)))
axes[3,3].set_title('oq vs oqd')

# ###############################

axes[4,0].plot(t,squeeze(oq.reshape(-1,12)))
axes[4,0].set_title('oq')

axes[4,1].plot(t,squeeze(oqd.reshape(-1,12)))
axes[4,1].set_title('oqd')

axes[4,2].plot(t,squeeze((oq - oqd).reshape(-1,12)))
axes[4,2].set_title('oq - oqd')

axes[4,3].plot(t,squeeze(zoq.reshape(-1,12)))
axes[4,3].set_title('zoq')

# ###############################

axes[5,0].plot(t,squeeze(TLd.reshape(-1,12)))
axes[5,0].set_title('TLd')

axes[5,1].plot(t,squeeze(u.reshape(-1,12)))
axes[5,1].set_title('FQ')

axes[5,2].plot(t,squeeze(u_paralel).reshape(-1,12))
axes[5,2].set_title('FQ paralel')

axes[5,3].plot(t,squeeze(u_perp).reshape(-1,12))
axes[5,3].set_title('FQ perpendicular')

# ###############################

axes[6,0].plot(t,squeeze(q.reshape(-1,12) - qd.reshape(-1,12)))
axes[6,0].set_title('q - qd')

axes[6,1].plot(t,squeeze(q.reshape(-1,12)))
axes[6,1].set_title('q')

axes[6,2].plot(t,squeeze(qd.reshape(-1,12)))
axes[6,2].set_title('qd')

axes[6,3].plot(t,squeeze(180/np.pi * np.arccos(mt(q) @ qd)))
axes[6,3].set_title('q error (deg)')

In [ ]:
fig, axes = subplots(1, 2, figsize=(12, 4), sharex=True)

rpy = 180/pi*unwrap(squeeze(rotationMatrixToEulerAngles(R).T), axis=0)
rpy_d = 180/pi*unwrap(squeeze(rotationMatrixToEulerAngles(Rd).T), axis=0)

axes[0].plot(t,rpy - rpy_d)
axes[0].legend(['roll error', 'pitch error', 'yaw error'])
axes[0].grid()

axes[1].plot(t,rpy)
axes[1].legend(['roll', 'pitch','yaw'])
axes[1].grid()

In [ ]:
# u_perp_d =  - (mQ * lq) * skew(q) @ (
#     + 1 / lq * skew(q) @ a 
#     + skew(dq) @ skew(q) @ oqd
#     + skew(q) @ skew(dq) @ oqd
#     + skew(q) @ skew(q) @ doqd
#     + 1 / (L_q*mL) * skew(dq) @ (X_pv @ pe + ve) * (mt(qd) @ R @ TLd)
#     + 1 / (L_q*mL) * skew(q) @ (X_pv @ ve + ae) * (mt(qd) @ R @ TLd)
#     + 1 / (L_q*mL) * skew(q) @ (X_pv @ pe + ve) * (mt(dqd) @ R @ TLd)
#     + 1 / (L_q*mL) * skew(q) @ (X_pv @ pe + ve) * (mt(qd) @ R @ skew(o) @ TLd)
#     + 1 / (L_q*mL) * skew(q) @ (X_pv @ pe + ve) * (mt(qd) @ R @ dTLd)
#     - L_a / L_q * skew(dq) @ R @ skew(rho).reshape(-1,n_cables,n_dims,3) @ inv(I) @ e_o * (mt(qd) @ R @ TLd) 
#     - L_a / L_q * skew(q) @ R @ skew(o) @ skew(rho).reshape(-1,n_cables,n_dims,3) @ inv(I) @ e_o * (mt(qd) @ R @ TLd) 
#     - L_a / L_q * skew(q) @ R @ skew(rho).reshape(-1,n_cables,n_dims,3) @ inv(I) @ de_o * (mt(qd) @ R @ TLd) 
#     - L_a / L_q * skew(q) @ R @ skew(rho).reshape(-1,n_cables,n_dims,3) @ inv(I) @ e_o * (mt(dqd) @ R @ TLd) 
#     - L_a / L_q * skew(q) @ R @ skew(rho).reshape(-1,n_cables,n_dims,3) @ inv(I) @ e_o * (mt(qd) @ R @ skew(o) @ TLd) 
#     - L_a / L_q * skew(q) @ R @ skew(rho).reshape(-1,n_cables,n_dims,3) @ inv(I) @ e_o * (mt(qd) @ R @ dTLd) 
#     + kq / L_q * mt(skew(dq)) @ qd
#     + kq / L_q * mt(skew(q)) @ dqd
#     + koq / L_oq * (zoq)
#     - L_q / L_oq * skew(q) @ qd
# )

fig, axes = subplots(9, 4, figsize=(10, 12), sharex=True)

all_axes = [ax.grid() for sublist in axes for ax in sublist]

fig.suptitle('u_perp decomposition', fontsize=16)

for i in range(n_cables):
    axes[0,i].plot(t,squeeze((1 / lq * skew(q) @ a )[:,i,...]))
    axes[0,0].set_ylabel('a')
    
    axes[1,i].plot(t,squeeze((
        + skew(dq) @ skew(q) @ oqd
        + skew(q) @ skew(dq) @ oqd
        + skew(q) @ skew(q) @ doqd
    )[:,i,...]))
    axes[1,0].set_ylabel('oqd / doqd')
    
    axes[2,i].plot(t,squeeze((
        + 1/mL * 1/L_q *(mt(dqd) @ TLd) * skew(q) @ (X_pv @ pe + ve)
        + 1/mL * 1/L_q *(mt(qd) @ dTLd) * skew(q) @ (X_pv @ pe + ve)
        + 1/mL * 1/L_q *(mt(qd) @ TLd) * skew(dq) @ (X_pv @ pe + ve)
        + 1/mL * 1/L_q *(mt(qd) @ TLd) * skew(q) @ (X_pv @ ve + ae)
    )[:,i,...]))
    axes[2,0].set_ylabel('x term\npe|ve')
    
    if(not POINT_MASS):
        axes[3,i].plot(t,squeeze((
        - L_a / L_q * skew(dq) @ R @ skew(rho).reshape(-1,n_cables,n_dims,3) @ inv(I) @ e_o * (mt(qd) @ R @ TLd) 
        - L_a / L_q * skew(q) @ R @ skew(o) @ skew(rho).reshape(-1,n_cables,n_dims,3) @ inv(I) @ e_o * (mt(qd) @ R @ TLd) 
        - L_a / L_q * skew(q) @ R @ skew(rho).reshape(-1,n_cables,n_dims,3) @ inv(I) @ de_o * (mt(qd) @ R @ TLd) 
        - L_a / L_q * skew(q) @ R @ skew(rho).reshape(-1,n_cables,n_dims,3) @ inv(I) @ e_o * (mt(dqd) @ R @ TLd) 
        - L_a / L_q * skew(q) @ R @ skew(rho).reshape(-1,n_cables,n_dims,3) @ inv(I) @ e_o * (mt(qd) @ R @ skew(o) @ TLd) 
        - L_a / L_q * skew(q) @ R @ skew(rho).reshape(-1,n_cables,n_dims,3) @ inv(I) @ e_o * (mt(qd) @ R @ dTLd) 
        )[:,i,...]))
        axes[3,0].set_ylabel('dq / dqd')
    
    axes[4,i].plot(t,squeeze((
        + kq / L_q * mt(skew(dq)) @ qd
        + kq / L_q * mt(skew(q)) @ dqd 
    )[:,i,...]))
    axes[4,0].set_ylabel('dq / dqd')
    
    axes[5,i].plot(t,squeeze((
        + koq / L_oq * zoq 
    )[:,i,...]))
    axes[5,0].set_ylabel('zoq')
    
    axes[6,i].plot(t,squeeze((
        - L_q / L_oq * skew(q) @ qd
    )[:,i,...]))
    axes[6,0].set_ylabel('q / qd')

    axes[7,i].plot(t,squeeze((
        - L_q / L_oq * skew(q) @ qd
    )[:,i,...]))
    axes[7,0].set_ylabel('q / qd')
    
    if(not POINT_MASS):
        axes[8,i].plot(t,squeeze((
            + 1 / lq * skew(q) @ a 
            + skew(dq) @ skew(q) @ oqd
            + skew(q) @ skew(dq) @ oqd
            + skew(q) @ skew(q) @ doqd
            + 1 / (L_q*mL) * skew(dq) @ (X_pv @ pe + ve) * (mt(qd) @ R @ TLd)
            + 1 / (L_q*mL) * skew(q) @ (X_pv @ ve + ae) * (mt(qd) @ R @ TLd)
            + 1 / (L_q*mL) * skew(q) @ (X_pv @ pe + ve) * (mt(dqd) @ R @ TLd)
            + 1 / (L_q*mL) * skew(q) @ (X_pv @ pe + ve) * (mt(qd) @ R @ skew(o) @ TLd)
            + 1 / (L_q*mL) * skew(q) @ (X_pv @ pe + ve) * (mt(qd) @ R @ dTLd)
            - L_a / L_q * skew(dq) @ R @ skew(rho).reshape(-1,n_cables,n_dims,3) @ inv(I) @ e_o * (mt(qd) @ R @ TLd) 
            - L_a / L_q * skew(q) @ R @ skew(o) @ skew(rho).reshape(-1,n_cables,n_dims,3) @ inv(I) @ e_o * (mt(qd) @ R @ TLd) 
            - L_a / L_q * skew(q) @ R @ skew(rho).reshape(-1,n_cables,n_dims,3) @ inv(I) @ de_o * (mt(qd) @ R @ TLd) 
            - L_a / L_q * skew(q) @ R @ skew(rho).reshape(-1,n_cables,n_dims,3) @ inv(I) @ e_o * (mt(dqd) @ R @ TLd) 
            - L_a / L_q * skew(q) @ R @ skew(rho).reshape(-1,n_cables,n_dims,3) @ inv(I) @ e_o * (mt(qd) @ R @ skew(o) @ TLd) 
            - L_a / L_q * skew(q) @ R @ skew(rho).reshape(-1,n_cables,n_dims,3) @ inv(I) @ e_o * (mt(qd) @ R @ dTLd) 
            + kq / L_q * mt(skew(dq)) @ qd
            + kq / L_q * mt(skew(q)) @ dqd
            + koq / L_oq * (zoq)
            - L_q / L_oq * skew(q) @ qd
        )[:,i,...]))
        axes[8,0].set_ylabel('u_perp')
    else:
        axes[8,i].plot(t,squeeze((
            + 1 / lq * skew(q) @ a 
            + skew(dq) @ skew(q) @ oqd
            + skew(q) @ skew(dq) @ oqd
            + skew(q) @ skew(q) @ doqd
            + 1 / (L_q*mL) * skew(dq) @ (X_pv @ pe + ve) * (mt(qd) @ R @ TLd)
            + 1 / (L_q*mL) * skew(q) @ (X_pv @ ve + ae) * (mt(qd) @ R @ TLd)
            + 1 / (L_q*mL) * skew(q) @ (X_pv @ pe + ve) * (mt(dqd) @ R @ TLd)
            + 1 / (L_q*mL) * skew(q) @ (X_pv @ pe + ve) * (mt(qd) @ R @ skew(o) @ TLd)
            + 1 / (L_q*mL) * skew(q) @ (X_pv @ pe + ve) * (mt(qd) @ R @ dTLd)
            + kq / L_q * mt(skew(dq)) @ qd
            + kq / L_q * mt(skew(q)) @ dqd
            + koq / L_oq * (zoq)
            - L_q / L_oq * skew(q) @ qd
        )[:,i,...]))
        axes[8,0].set_ylabel('u_perp')
    
all_axes = [ax for sublist in axes for ax in sublist]
y_lim_equalize(all_axes)

In [ ]:
# u_paralel = mu + mQ * lq * norm(oq, axis=-2, keepdims=True)**2 * q + mQ * q @ mt(q) @ a

fig, axes = subplots(4, 4, figsize=(10, 8), sharex=True)

all_axes = [ax.grid() for sublist in axes for ax in sublist]

fig.suptitle('u_paralel decomposition', fontsize=16)

for i in range(n_cables):
    axes[0,i].plot(t,squeeze((
        mu
    )[:,i,:]))
    axes[0,0].set_ylabel('mu')
    
    axes[1,i].plot(t,squeeze((
        mQ * lq * norm(oq, axis=-2, keepdims=True)**2 * q
    )[:,i,:]))
    axes[1,0].set_ylabel('q / oq')
    
    axes[2,i].plot(t,squeeze((
        mQ * q @ mt(q) @ a
    )[:,i,:]))
    axes[2,0].set_ylabel('a')
    
    axes[3,i].plot(t,squeeze((
        u_paralel
    )[:,i,:]))
    axes[3,0].set_ylabel('u_paralel')
    
all_axes = [ax for sublist in axes for ax in sublist]
y_lim_equalize(all_axes)

### TLd vs TL

In [ ]:
fig, axes = subplots(2, 2, figsize=(10, 8), sharex=True)

all_axes = [ax.grid() for sublist in axes for ax in sublist]

all_axes = [ax for sublist in axes for ax in sublist]

fig.suptitle('TL vs TLd', fontsize=16)

for i in range(n_cables):
#     all_axes[i].plot(t,squeeze(TL[:,i,0:2,0]))
    all_axes[i].plot(t,squeeze(TLd[:,i,0:2,0]))
    all_axes[i].legend([f'TL{i}_0',f'TL{i}_1',f'TLd{i}_0',f'TLd{i}_1'])

all_axes = [ax for sublist in axes for ax in sublist]
y_lim_equalize(all_axes)

In [ ]:
# u_paralel = mu + mQ * lq * norm(oq, axis=-2, keepdims=True)**2 * q + mQ * q @ mt(q) @ a

fig, axes = subplots(2, 2, figsize=(10, 8), sharex=True)

all_axes = [ax.grid() for sublist in axes for ax in sublist]

all_axes = [ax for sublist in axes for ax in sublist]

fig.suptitle('mu vs mud', fontsize=16)

for i in range(n_cables):
    all_axes[i].plot(t,squeeze(mu[:,i,0:2,0]))
    all_axes[i].plot(t,squeeze(mud[:,i,0:2,0]))

legend(['mu_x','mu_y','mud_x','mud_y'])
    
all_axes = [ax for sublist in axes for ax in sublist]
y_lim_equalize(all_axes)

### q vs qd

In [ ]:
# u_paralel = mu + mQ * lq * norm(oq, axis=-2, keepdims=True)**2 * q + mQ * q @ mt(q) @ a

fig, axes = subplots(2, 2, figsize=(10, 8), sharex=True)

all_axes = [ax.grid() for sublist in axes for ax in sublist]

all_axes = [ax for sublist in axes for ax in sublist]

fig.suptitle('q vs qd', fontsize=16)

for i in range(n_cables):
    all_axes[i].plot(t,squeeze(q[:,i,0:2,0]))
    all_axes[i].plot(t,squeeze(qd[:,i,0:2,0]))

legend(['q_x','q_y','qd_x','qd_y'])

all_axes = [ax for sublist in axes for ax in sublist]
y_lim_equalize(all_axes)

### qR vs qRd

In [ ]:
r3 = qR @ e3
r3d = qRd @ e3
    
fig, axes = subplots(2, 2, figsize=(10, 8), sharex=True)

all_axes = [ax.grid() for sublist in axes for ax in sublist]

all_axes = [ax for sublist in axes for ax in sublist]

fig.suptitle('r3 vs r3d', fontsize=16)

for i in range(n_cables):
    all_axes[i].plot(t,squeeze(r3[:,0,0:2,0]))
    all_axes[i].plot(t,squeeze(r3d[:,0,0:2,0]))
    # plot(t,squeeze(qT[:,0,0,0]))

legend(['r3_x','r3_y','r3d_x','r3d_y'])

all_axes = [ax for sublist in axes for ax in sublist]
y_lim_equalize(all_axes)

In [ ]:
plot(t,squeeze(qT[:,:,...]))
grid()
title('T')

In [ ]:
print("Offset in r3 tracking r3d")

for i in range(3):
    from scipy.signal import  correlate
    tmp = (correlate(r3[:,i,0,0],r3d[:,i,0,0]))
    # plot(r3[:,i,0,0])
    # plot(r3d[:,i,0,0])
    midpoint = len(tmp)//2
    delta = tmp[len(tmp)//2:-1].argmax() / 50
    print(delta)

In [ ]:
# u_paralel = mu + mQ * lq * norm(oq, axis=-2, keepdims=True)**2 * q + mQ * q @ mt(q) @ a

fig, axes = subplots(1, 2, figsize=(12, 4), sharex=True)

axes[0].plot(t,squeeze(F)[:,0:2])
axes[0].plot(t,squeeze(Fd)[:,0:2])
axes[0].grid()
axes[0].set_title('F vs Fd')

if (not POINT_MASS):
    axes[1].set_title('M vs Md')
    axes[1].plot(t,squeeze(M)[:,0:2])
    axes[1].plot(t,squeeze(Md)[:,0:2])
    axes[1].grid()

In [ ]:
if (not POINT_MASS):
    fig, axes = subplots(5, 4, figsize=(10, 8), sharex=True)

    all_axes = [ax.grid() for sublist in axes for ax in sublist]

    fig.suptitle('Md decomposition', fontsize=16)

    for i in range(n_cables):
        axes[0,i].plot(t,squeeze(I @ (- kr*e_R)))
        axes[0,0].set_ylabel('e_R')

        axes[1,i].plot(t,squeeze(I @ (- ko*e_o)))
        axes[1,0].set_ylabel('e_o')

        axes[2,i].plot(t,squeeze(I @ taud))
        axes[2,0].set_ylabel('taud')

        axes[3,i].plot(t,squeeze(skew(o)@I@o))
        axes[3,0].set_ylabel('o^2')

        axes[4,i].plot(t,squeeze(Md))
        axes[4,0].set_ylabel('Md')

    all_axes = [ax for sublist in axes for ax in sublist]
    y_lim_equalize(all_axes)

In [ ]:
if (not POINT_MASS):
    # u_paralel = mu + mQ * lq * norm(oq, axis=-2, keepdims=True)**2 * q + mQ * q @ mt(q) @ a

    fig, axes = subplots(2, 2, figsize=(10, 8), sharex=True)

    all_axes = [ax.grid() for sublist in axes for ax in sublist]

    all_axes = [ax for sublist in axes for ax in sublist]

    fig.suptitle('M decomposition', fontsize=16)

    all_axes[0].plot(t,squeeze(M))
    all_axes[1].plot(t,squeeze(I @ ( - kr*e_R)))
    all_axes[2].plot(t,squeeze(I @ ( - ko*e_o)))
    all_axes[3].plot(t,squeeze(skew(o)@I@o))

    all_axes[0].set_title('M')
    all_axes[1].set_title('e_R')
    all_axes[2].set_title('e_o')
    all_axes[3].set_title('o')

    all_axes = [ax for sublist in axes for ax in sublist]
    y_lim_equalize(all_axes)

In [ ]:
# fin

### Graphics definitions

In [ ]:
tf = t[-1]
t_step = 0.02

In [ ]:
load = array([ [ length/2,  width/2, height/2],
                [-length/2,  width/2, height/2],
                [-length/2, -width/2, height/2],
                [ length/2, -width/2, height/2],
                [ length/2,  width/2, -height/2],
                [-length/2,  width/2, -height/2],
                [-length/2, -width/2, -height/2],
                [ length/2, -width/2, -height/2]] ).T

def load_edges_at_pose(p=array([0,0,0]), R=eye(3)):
    # https://stackoverflow.com/questions/44881885/python-draw-3d-cube/49766400#49766400

    p = p.ravel()
    
    cube_definition = [
        p + R @ load[:,0], 
        p + R @ load[:,1], 
        p + R @ load[:,3], 
        p + R @ load[:,4]
    ]

    cube_definition_array = [
        np.array(list(item))
        for item in cube_definition
    ]

    points = []
    points += cube_definition_array
    vectors = [
        cube_definition_array[1] - cube_definition_array[0],
        cube_definition_array[2] - cube_definition_array[0],
        cube_definition_array[3] - cube_definition_array[0]
    ]

    points += [cube_definition_array[0] + vectors[0] + vectors[1]]
    points += [cube_definition_array[0] + vectors[0] + vectors[2]]
    points += [cube_definition_array[0] + vectors[1] + vectors[2]]
    points += [cube_definition_array[0] + vectors[0] + vectors[1] + vectors[2]]

    points = np.array(points)

    edges = [
        [points[0], points[3], points[5], points[1]],
        [points[1], points[5], points[7], points[4]],
        [points[4], points[2], points[6], points[7]],
        [points[2], points[6], points[3], points[0]],
        [points[0], points[2], points[4], points[1]],
        [points[3], points[6], points[7], points[5]]
    ]
    
    return edges


def quadrotor_edges_at_pose(p=array([0,0,0]), R=eye(3)):

    p = p.ravel()
    
    length = 0.2
    width = 0.2
    height = 0.05

    quad = array([ [ length/2,  width/2, height/2],
                    [-length/2,  width/2, height/2],
                    [-length/2, -width/2, height/2],
                    [ length/2, -width/2, height/2],
                    [ length/2,  width/2, -height/2],
                    [-length/2,  width/2, -height/2],
                    [-length/2, -width/2, -height/2],
                    [ length/2, -width/2, -height/2]] ).T    

    # https://stackoverflow.com/questions/44881885/python-draw-3d-cube/49766400#49766400

    cube_definition = [
        p + R @ quad[:,0], 
        p + R @ quad[:,1], 
        p + R @ quad[:,3], 
        p + R @ quad[:,4]
    ]

    cube_definition_array = [
        np.array(list(item))
        for item in cube_definition
    ]

    points = []
    points += cube_definition_array
    vectors = [
        cube_definition_array[1] - cube_definition_array[0],
        cube_definition_array[2] - cube_definition_array[0],
        cube_definition_array[3] - cube_definition_array[0]
    ]

    points += [cube_definition_array[0] + vectors[0] + vectors[1]]
    points += [cube_definition_array[0] + vectors[0] + vectors[2]]
    points += [cube_definition_array[0] + vectors[1] + vectors[2]]
    points += [cube_definition_array[0] + vectors[0] + vectors[1] + vectors[2]]

    points = np.array(points)

    edges = [
        [points[0], points[3], points[5], points[1]],
        [points[1], points[5], points[7], points[4]],
        [points[4], points[2], points[6], points[7]],
        [points[2], points[6], points[3], points[0]],
        [points[0], points[2], points[4], points[1]],
        [points[3], points[6], points[7], points[5]]
    ]
    
    return edges


def draw_load(ax,p=zeros(3),R=eye(3)):
    
    p = p.reshape(3,1)
    R = R.reshape(3,3)
    
    edges = load_edges_at_pose(p, R)
    faces = Poly3DCollection(edges, linewidths=1, edgecolors='k')

    # # Color all faces alike
    # colors = (0,0,1,0.2)

    # Color front face differently
    colors = len(edges)*[(0,0,1,0.2)]
    colors[3] = (1,1,1,0.6)
    
    faces.set_facecolor(colors)
    
    ax.add_collection3d(faces)

    
def draw_ghost_load(ax,p=zeros(3),R=eye(3)):
    
    p = p.reshape(3,1)
    R = R.reshape(3,3)
    
    edges = load_edges_at_pose(p, R)
    faces = Poly3DCollection(edges, linewidths=0.25, edgecolors='k')
    faces.set_facecolor((0,0,1,0.05))
    ax.add_collection3d(faces)

    
    
def draw_q(ax, q, p=zeros([3,1]), R=eye(3), color=(0,1,1)):
    
    # Attachement points for the cables
    rho_inertial = R @ rho + p
    
    # Compute cable position based on direction by TL 
    q = q / norm(q,axis=-2)[..., np.newaxis,:]
    
    # Draw on 'ax'
    ax.quiver(array(rho_inertial[:,0]).astype(float), 
              array(rho_inertial[:,1]).astype(float), 
              array(rho_inertial[:,2]).astype(float), 
              array(-(q*lq)[:,0]).astype(float), 
              array(-(q*lq)[:,1]).astype(float), 
              array(-(q*lq)[:,2]).astype(float),
              color=color,
              arrow_length_ratio=0,
              )
    
def draw_TL(ax, B_TL, p=zeros([3,1]), R=eye(3), color=(0,1,1)):
    
    # Attachement points for the cables
    rho_inertial = R @ rho + p
    
    # Compute cable position based on direction by TL 
    q = R @ ( - B_TL / norm(B_TL,axis=-2)[..., np.newaxis,:])
    
    # Draw on 'ax'
    ax.quiver(array(rho_inertial[:,0]).astype(float), 
              array(rho_inertial[:,1]).astype(float), 
              array(rho_inertial[:,2]).astype(float), 
              array(-(q*lq)[:,0]).astype(float), 
              array(-(q*lq)[:,1]).astype(float), 
              array(-(q*lq)[:,2]).astype(float),
              color=color,
              arrow_length_ratio=0,
              )
    
    
def draw_quadrotor(ax,p=zeros([3,1]),R=eye(3), T=0, color=(0,1,1), alpha = 0.5):
    
    p = p.reshape(3,1)
    R = R.reshape(3,3)
    
    c = hstack([color, alpha])
    
    edges = quadrotor_edges_at_pose(p, R)
    faces = Poly3DCollection(edges, linewidths=0.5, edgecolors='k')
    faces.set_facecolor(c)
    ax.add_collection3d(faces)
    
    # TODO: Move to "draw quadrotor"
    r3 = R @ e3
    ax.quiver3D(array(p[0]).astype(float), 
              array(p[1]).astype(float), 
              array(p[2]).astype(float), 
              array(-T*r3[0]).astype(float), 
              array(-T*r3[1]).astype(float), 
              array(-T*r3[2]).astype(float),
              color=color,
              arrow_length_ratio=0,
              length=0.05
              )


def draw_quadrotors_TL(ax, TL_matrix, p=zeros([3,1]), R=eye(3), color=(0,1,1), alpha=0.5):

    c = hstack([color, alpha])
    
    rho_inertial = R @ rho + p
    
    cable_matrix = R @ (-TL_matrix / norm(TL_matrix,axis=-2)[..., np.newaxis,:]) * lq

    p_quads = rho_inertial - cable_matrix
    
    # Gravity has same shape as TL_matrix
    s = asarray(TL_matrix.shape)
    s[-2] = 1
    
    gravity = g*tile(e3,s)
    
    I_T = (R @ TL_matrix - mQ*gravity)
    
    qT = norm(I_T,axis=-2, keepdims=True)
    
    e1_tile = tile(e1,qT.shape)
    
    r3 = -I_T
    r2 = cross(r3,e1_tile,axis=-2)
    r1 = cross(r2,r3,axis=-2)
    
    Rq = concatenate([r1,r2,r3],axis=-1)
    Rq = Rq / norm(Rq,axis=-2,keepdims=True)
    
    for i in range(p_quads.shape[0]):
        draw_quadrotor(ax, p_quads[i], Rq[i], T=qT[i], color=color)

    return




def draw_quadrotors(ax, qp, qR, qT, color=(0,1,1), alpha=0.5):

    for i in range(len(qp)):        
        draw_quadrotor(ax, qp[i], qR[i], qT[i], color=color)
    
    return

In [ ]:
if (POINT_MASS):
    (TLd_min, TLd_opt) = compute_TL(t,Fd)
else:
    (TLd_min, TLd_opt, B_TLd_min, B_TLd_opt_body) = compute_vector_TL_restrictions_body(t,B_FdMd,R)
    (TLd_min, TLd_opt, B_TLd_min, B_TLd_opt_inertial) = compute_vector_TL_restrictions_inertial(t,B_FdMd,R)

In [ ]:
if (POINT_MASS):
    import ipywidgets as widgets
    from ipywidgets import interact

    colors = [ array([1,0,0]),
            array([0,1,0]),
            array([0,0,1]),
            array([1,1,0]),
            array([1,0,1]),
            array([0,1,1]),
            array([1,1,1]),
            ]

    def f(t_frame,el,az):

        ## New figure

        # fig = figure(figsize=(30, 30))

        fig, axs = subplots(1,2,subplot_kw=dict(projection='3d'),figsize=(20, 10))
        fig.subplots_adjust(wspace=-0.1)

        # Recover index (of differential eq. solution) for the current time
        i = argmax(t>=t_frame)
        ti = t[i]

        Ri = eye(3)

        fig.suptitle(f't = {ti:0.2}', fontsize=24)

        # TL matrix for the minimal solution
        TL_min_matrix = TLd_min[i]

        # TL matrix for the solution with repulsion
        TL_complete_matrix = (TLd_min[i] + delta_TLd[i])

        # TL matrix for the optimization solution
        TL_opt_matrix = TLd_opt[i]

        ### First subfigure - Inertial view

        ax = axs[0]

        ax.set_title("Inertial view")

        # Minimal solution

        draw_TL(ax, TL_min_matrix, p[i], Ri, color=colors[0])
        draw_quadrotors_TL(ax, TL_min_matrix, p[i], Ri, color=colors[0])

        # Repulsion solution

        draw_TL(ax, TL_complete_matrix, p[i], Ri, color=colors[1])
        draw_quadrotors_TL(ax, TL_complete_matrix, p[i], Ri, color=colors[1])

        # Optimization solution

        draw_TL(ax, TL_opt_matrix, p[i], Ri, color=colors[2])
        draw_quadrotors_TL(ax, TL_opt_matrix, p[i], Ri, color=colors[2])

        draw_load(ax, p[i], Ri)

        draw_ghost_load(ax, pr(ti), eye(3))

        draw_q(ax, q[i], p[i], R=Ri, color=colors[4])
        draw_quadrotors(ax, qp[i], qR[i], qT[i], color=colors[4])

        ax.plot(pr(t)[:,0,0].ravel(),
                pr(t)[:,0,1].ravel(),
                pr(t)[:,0,2].ravel())

    #     ax.auto_scale_xyz(3,3,1)
    #     ax.autoscale_view()

        ax.view_init(elev=el, azim=az)

        ax.set_xlim3d(-2,2)
        ax.set_ylim3d(-2,2)
        ax.set_zlim3d(-2,2)

    #     ax.axis('equal')
    #     ax.axis('scaled') 
    #     ax.set_aspect('equal')

        ax.invert_zaxis()

    #     ### Second subfigure - Body view

        ax = axs[1]

        ax.set_title("'Same yaw' view")

        v = rotationMatrixToEulerAngles(Ri.reshape(3,3))
        v[2] = 0
        Ri = eulerAnglesToRotationMatrix(v).reshape(1,3,3)

        draw_TL(ax, TL_min_matrix, R=Ri, color=colors[0])
        draw_quadrotors_TL(ax, TL_min_matrix, R=Ri, color=colors[0])

        # Repulsion solution

        draw_TL(ax, TL_complete_matrix, R=Ri, color=colors[1])
        draw_quadrotors_TL(ax, TL_complete_matrix, R=Ri, color=colors[1])

        # Optimization solution

        draw_TL(ax, TL_opt_matrix, R=Ri, color=colors[2])
        draw_quadrotors_TL(ax, TL_opt_matrix, R=Ri, color=colors[2])

        draw_load(ax, R=Ri)

        draw_q(ax, q[i], R=Ri, color=colors[4])
        draw_quadrotors(ax, (qp[i] - p[i]), qR[i], qT[i], color=colors[4])

        ax.set_xlim3d(-1,1)
        ax.set_ylim3d(-1,1)
        ax.set_zlim3d(-1.5, 0.5)

        ax.view_init(elev=el, azim=az+180)

    #     ax.set_aspect('equal')

        ax.invert_zaxis()

        import matplotlib.patches as mpatches
        import matplotlib.pyplot as plt

        plt.legend(handles=[mpatches.Patch(color=colors[0], label='TLd min'), 
                           mpatches.Patch(color=colors[1], label='TLd min + delta_TLd'),
                           mpatches.Patch(color=colors[2], label='TLd opt'),
                           mpatches.Patch(color=colors[4], label='TLd actual')])
        return

    interact(f, 
             t_frame=widgets.FloatSlider(min=0,max=tf-0.01,value=0,step=0.01),
                 el=widgets.FloatSlider(min=0,max=90,value=20),
                 az=widgets.FloatSlider(min=0,max=360,value=20),
             continuous_update=True)
else:
    import ipywidgets as widgets
    from ipywidgets import interact

    colors = [ array([1,0,0]),
            array([0,1,0]),
            array([0,0,1]),
            array([1,1,0]),
            array([1,0,1]),
            array([0,1,1]),
            array([1,1,1]),
            ]

    def f(t_frame,el,az):

        ## New figure

        # fig = figure(figsize=(30, 30))

        fig, axs = subplots(1,2,subplot_kw=dict(projection='3d'),figsize=(20, 10))
        fig.subplots_adjust(wspace=-0.1)

        # Recover index (of differential eq. solution) for the current time
        i = argmax(t>=t_frame)
        ti = t[i]

        fig.suptitle(f't = {ti}', fontsize=24)

        Ri = R[i]

        # TL matrix for the minimal solution
        TL_min_matrix = B_TLd_min[i]

        # TL matrix for the solution with repulsion
        TL_complete_matrix = (B_TLd_min[i] + delta_TLd[i])

        # TL matrix for the optimization solution - body
        TL_opt_body_matrix = B_TLd_opt_body[i]

        # TL matrix for the optimization solution - inertial
        TL_opt_inertial_matrix = B_TLd_opt_inertial[i]

        ### First subfigure - Inertial view

        ax = axs[0]

        ax.set_title("Inertial view")

        # Minimal solution

        draw_TL(ax, TL_min_matrix, p[i], Ri, color=colors[0])
        draw_quadrotors_TL(ax, TL_min_matrix, p[i], Ri, color=colors[0])

        # Repulsion solution

        draw_TL(ax, TL_complete_matrix, p[i], Ri, color=colors[1])
        draw_quadrotors_TL(ax, TL_complete_matrix, p[i], Ri, color=colors[1])

        # Optimization solution - body

        draw_TL(ax, TL_opt_body_matrix, p[i], Ri, color=colors[2])
        draw_quadrotors_TL(ax, TL_opt_body_matrix, p[i], Ri, color=colors[2])

        # Optimization solution - inertial

        draw_TL(ax, TL_opt_inertial_matrix, p[i], Ri, color=colors[3])
        draw_quadrotors_TL(ax, TL_opt_inertial_matrix, p[i], Ri, color=colors[3])

        draw_load(ax, p[i], Ri)

        draw_ghost_load(ax, pr(ti), Rr(ti))

        draw_q(ax, q[i], p[i], R=Ri, color=colors[4])
        draw_quadrotors(ax, qp[i], qR[i], qT[i], color=colors[4])

        ax.plot(pr(t)[:,0,0].ravel(),
                pr(t)[:,0,1].ravel(),
                pr(t)[:,0,2].ravel())

    #     ax.auto_scale_xyz(3,3,1)
    #     ax.autoscale_view()

        ax.view_init(elev=el, azim=az)

        ax.set_xlim3d(-2,2)
        ax.set_ylim3d(-2,2)
        ax.set_zlim3d(-2,2)

    #     ax.axis('equal')
    #     ax.axis('scaled') 
    #     ax.set_aspect('equal')

        ax.invert_zaxis()

    #     ### Second subfigure - Body view

        ax = axs[1]

        ax.set_title("'Same yaw' view")

        v = rotationMatrixToEulerAngles(Ri.reshape(3,3))
        v[2] = 0
        Ri = eulerAnglesToRotationMatrix(v).reshape(1,3,3)

        draw_TL(ax, TL_min_matrix, R=Ri, color=colors[0])
        draw_quadrotors_TL(ax, TL_min_matrix, R=Ri, color=colors[0])

        # Repulsion solution

        draw_TL(ax, TL_complete_matrix, R=Ri, color=colors[1])
        draw_quadrotors_TL(ax, TL_complete_matrix, R=Ri, color=colors[1])

        # Optimization solution - body

        draw_TL(ax, TL_opt_body_matrix, R=Ri, color=colors[2])
        draw_quadrotors_TL(ax, TL_opt_body_matrix, R=Ri, color=colors[2])

        # Optimization solution - inertial

        draw_TL(ax, TL_opt_inertial_matrix, R=Ri, color=colors[3])
        draw_quadrotors_TL(ax, TL_opt_inertial_matrix, R=Ri, color=colors[3])

        draw_load(ax, R=Ri)

        draw_q(ax, q[i], R=Ri, color=colors[4])
        draw_quadrotors(ax, Ri @ mt(R[i]) @ (qp[i] - p[i]), qR[i], qT[i], color=colors[4])

        ax.set_xlim3d(-1,1)
        ax.set_ylim3d(-1,1)
        ax.set_zlim3d(-1.5, 0.5)

        ax.view_init(elev=el, azim=az+180)

    #     ax.set_aspect('equal')

        ax.invert_zaxis()

        return

    a = interact(f, 
             t_frame=widgets.FloatSlider(min=0,max=tf-0.01,value=0,step=0.01),
                 el=widgets.FloatSlider(min=0,max=90,value=20),
                 az=widgets.FloatSlider(min=0,max=360,value=20),
             continuous_update=False)

In [ ]:
if( POINT_MASS):
    from IPython.display import HTML

    fig, axs = subplots(1,2,subplot_kw=dict(projection='3d'),figsize=(18, 6))
    fig.subplots_adjust(wspace=-0.1)


    el = 20
    az = 20

    def update_frame(i):

        axs[0].clear()
        axs[1].clear()

        ax = axs[0]

    #     ax.set_xlim3d(-15,15)
    #     ax.set_ylim3d(-15,15)
    #     ax.set_zlim3d(-10,20)

        ax.set_xlim3d(-3,3)
        ax.set_ylim3d(-3,3)
        ax.set_zlim3d(-4.5,1.5)

        ax.view_init(elev=el, azim=az)
    #     ax.set_aspect('equal')
        ax.invert_zaxis()

        ax.set_xlabel('X', size=16)
        ax.set_ylabel('Y', size=16)
        ax.set_zlabel('Z', size=16)

        ax = axs[1]

        ax.set_xlim3d(-1.0,1.0)
        ax.set_ylim3d(-1.0,1.0)
        ax.set_zlim3d(-2, 0)

        ax.view_init(elev=el, azim=az)
    #     ax.set_aspect('equal')
        ax.invert_zaxis()

        ax.set_xlabel('X', size=16)
        ax.set_ylabel('Y', size=16)
        ax.set_zlabel('Z', size=16)


        ti = t[i]

        fig.suptitle(f't = {ti:.2f}', fontsize=16)

        Ri = eye(3)

        # TLd matrix for the minimal solution
        TLd_min_matrix = TLd_min[i]

        # TL matrix for the solution with repulsion
        TLd_complete_matrix = (TLd_min[i] + delta_TLd[i])

        # TL matrix for the optimization solution - body
        TLd_opt_matrix = TLd_opt[i]

        ### First subfigure - Inertial view

        ax = axs[0]

        ax.set_title("Inertial view")

        # Minimal solution

        draw_TL(ax, TLd_min_matrix, p[i], Ri, color=colors[0])
        draw_quadrotors_TL(ax, TLd_min_matrix, p[i], Ri, color=colors[0])

        # Repulsion solution

        draw_TL(ax, TLd_complete_matrix, p[i], Ri, color=colors[1])
        draw_quadrotors_TL(ax, TLd_complete_matrix, p[i], Ri, color=colors[1])

        # Optimization solution

        draw_TL(ax, TLd_opt_matrix, p[i], Ri, color=colors[2])
        draw_quadrotors_TL(ax, TLd_opt_matrix, p[i], Ri, color=colors[2])

        draw_load(ax, p[i], Ri)

        draw_ghost_load(ax, pr(ti), eye(3))

        draw_q(ax, q[i], p[i], Ri, color=colors[4])

        draw_quadrotors(ax, qp[i], qR[i], qT[i],color=colors[4])

        ax.plot(pr(t)[:,0,0].ravel(),
                pr(t)[:,0,1].ravel(),
                pr(t)[:,0,2].ravel())

        ### Second subfigure - Zoom view

        ax = axs[1]

        ax.set_title("Zoom view")

        draw_TL(ax, TLd_min_matrix, R=R[i], color=colors[0])
        draw_quadrotors_TL(ax, TLd_min_matrix, R=R[i], color=colors[0])

        # Repulsion solution

        draw_TL(ax, TLd_complete_matrix, R=R[i], color=colors[1])
        draw_quadrotors_TL(ax, TLd_complete_matrix, R=R[i], color=colors[1])

        # Optimization solution - body

        draw_TL(ax, TLd_opt_matrix, R=R[i], color=colors[2])
        draw_quadrotors_TL(ax, TLd_opt_matrix, R=R[i], color=colors[2])

        # Draw simulated quadrotors
        draw_q(ax, q[i], R=R[i], color=colors[4])
        draw_quadrotors(ax, qp[i] - p[i], qR[i], qT[i], color=colors[4])

        draw_load(ax, R=R[i])

        return

    anim = FuncAnimation(fig, update_frame,
                                   frames=len(t), interval=1000*t_step, 
                                   blit=False)    
else:
    from IPython.display import HTML

    # fig, axs = subplots(1,3,subplot_kw=dict(projection='3d'),figsize=(18, 6))
    fig, axs = subplots(1,3,subplot_kw=dict(projection='3d'),figsize=(12, 4))
    fig.subplots_adjust(wspace=-0.1)


    el = 20
    az = 20

    def update_frame(i):

        axs[0].clear()
        axs[1].clear()
        axs[2].clear()

        ax = axs[0]

    #     ax.set_xlim3d(-15,15)
    #     ax.set_ylim3d(-15,15)
    #     ax.set_zlim3d(-10,20)

        ax.set_xlim3d(-3,3)
        ax.set_ylim3d(-3,3)
        ax.set_zlim3d(-3,3)

        ax.view_init(elev=el, azim=az)
    #     ax.set_aspect('equal')
        ax.invert_zaxis()

        ax.set_xlabel('X', size=16)
        ax.set_ylabel('Y', size=16)
        ax.set_zlabel('Z', size=16)

        ax = axs[1]

        ax.set_xlim3d(-1.0,1.0)
        ax.set_ylim3d(-1.0,1.0)
        ax.set_zlim3d(-2, 0)

        ax.view_init(elev=el, azim=az+180)
    #     ax.set_aspect('equal')
        ax.invert_zaxis()

        ax.set_xlabel('X', size=16)
        ax.set_ylabel('Y', size=16)
        ax.set_zlabel('Z', size=16)


        ax = axs[2]

        ax.set_xlim3d(-1.0,1.0)
        ax.set_ylim3d(-1.0,1.0)
        ax.set_zlim3d(-2, 0)

        ax.view_init(elev=el, azim=az)
    #     ax.set_aspect('equal')
        ax.invert_zaxis()

        ax.set_xlabel('X', size=16)
        ax.set_ylabel('Y', size=16)
        ax.set_zlabel('Z', size=16)


        ti = t[i]

        fig.suptitle(f't = {ti:.2f}', fontsize=16)

        Ri = R[i]

        # TLd matrix for the minimal solution
        TLd_min_matrix = B_TLd_min[i]

        # TL matrix for the solution with repulsion
        TLd_complete_matrix = (B_TLd_min[i] + delta_TLd[i])

        # TL matrix for the optimization solution - body
        TLd_opt_body_matrix = B_TLd_opt_body[i]

        # TL matrix for the optimization solution - inertial
        TLd_opt_inertial_matrix = B_TLd_opt_inertial[i]




        ### First subfigure - Inertial view

        ax = axs[0]

        ax.set_title("Inertial view")

        # Minimal solution

        draw_TL(ax, TLd_min_matrix, p[i], Ri, color=colors[0])
        draw_quadrotors_TL(ax, TLd_min_matrix, p[i], Ri, color=colors[0])

        # Repulsion solution

        draw_TL(ax, TLd_complete_matrix, p[i], Ri, color=colors[1])
        draw_quadrotors_TL(ax, TLd_complete_matrix, p[i], Ri, color=colors[1])

        # Optimization solution - body

        draw_TL(ax, TLd_opt_body_matrix, p[i], Ri, color=colors[2])
        draw_quadrotors_TL(ax, TLd_opt_body_matrix, p[i], Ri, color=colors[2])

        # Optimization solution - inertial

        draw_TL(ax, TLd_opt_inertial_matrix, p[i], Ri, color=colors[3])
        draw_quadrotors_TL(ax, TLd_opt_inertial_matrix, p[i], Ri, color=colors[3])

        draw_load(ax, p[i], Ri)

        draw_ghost_load(ax, pr(ti), Rr(ti))

        draw_q(ax, q[i], p[i], Ri, color=colors[4])

        draw_quadrotors(ax, qp[i], qR[i], qT[i],color=colors[4])

        ax.plot(pr(t)[:,0,0].ravel(),
                pr(t)[:,0,1].ravel(),
                pr(t)[:,0,2].ravel())



        ### Second subfigure - Body view

        ax = axs[1]

        ax.set_title("'Same yaw' view")

        v = rotationMatrixToEulerAngles(Ri.reshape(3,3))
        v[2] = 0
        Ri = eulerAnglesToRotationMatrix(v).reshape(1,3,3)

        draw_TL(ax, TLd_min_matrix, R=Ri, color=colors[0])
        draw_quadrotors_TL(ax, TLd_min_matrix, R=Ri, color=colors[0])

        # Repulsion solution

        draw_TL(ax, TLd_complete_matrix, R=Ri, color=colors[1])
        draw_quadrotors_TL(ax, TLd_complete_matrix, R=Ri, color=colors[1])

        # Optimization solution - body

        draw_TL(ax, TLd_opt_body_matrix, R=Ri, color=colors[2])
        draw_quadrotors_TL(ax, TLd_opt_body_matrix, R=Ri, color=colors[2])

        # Optimization solution - inertial

        draw_TL(ax, TLd_opt_inertial_matrix, R=Ri, color=colors[3])
        draw_quadrotors_TL(ax, TLd_opt_inertial_matrix, R=Ri, color=colors[3])

        draw_load(ax, R=Ri)

        draw_q(ax, Ri @ mt(R[i]) @ q[i], R=Ri, color=colors[4])
        draw_quadrotors(ax, Ri @ mt(R[i]) @ (qp[i] - p[i]), qR[i], qT[i], color=colors[4])


        ### Third subfigure - Body view

        ax = axs[2]

        ax.set_title("Zoom view")

        draw_TL(ax, TLd_min_matrix, R=R[i], color=colors[0])
        draw_quadrotors_TL(ax, TLd_min_matrix, R=R[i], color=colors[0])

        # Repulsion solution

        draw_TL(ax, TLd_complete_matrix, R=R[i], color=colors[1])
        draw_quadrotors_TL(ax, TLd_complete_matrix, R=R[i], color=colors[1])

        # Optimization solution - body

        draw_TL(ax, TLd_opt_body_matrix, R=R[i], color=colors[2])
        draw_quadrotors_TL(ax, TLd_opt_body_matrix, R=R[i], color=colors[2])

        # Optimization solution - inertial

        draw_TL(ax, TLd_opt_inertial_matrix, R=R[i], color=colors[3])
        draw_quadrotors_TL(ax, TLd_opt_inertial_matrix, R=R[i], color=colors[3])

        # Draw simulated quadrotors
        draw_q(ax, q[i], R=R[i], color=colors[4])
        draw_quadrotors(ax, qp[i] - p[i], qR[i], qT[i], color=colors[4])

        draw_load(ax, R=R[i])

        return

    anim = FuncAnimation(fig, update_frame,
                                   frames=len(t), interval=1000*t_step, 
                                   blit=False)

In [ ]:
HTML(anim.to_html5_video())
# HTML(anim.to_jshtml())